# Análisis de precios de propiedades en Colombia

Este notebook implementa el análisis de los datos disponibles en Kaggle "Colombia Housing Properties Price" disponibles en el sitio de Kaggle:

https://www.kaggle.com/julianusugaortiz/colombia-housing-properties-price/download

<img src="dataset-cover.jpg" width="1000" height="400">

El dataset incluye precios reales de propiedades inmobiliarias en colombia. Para tener disponible nuestro dataset usaremos
la librería kaggle que nos permite descargar los dataset de Kaggle. Tenga en cuenta que para esto necesitará suministrar un token de autorización para el ingreso a su cuenta, descargándolo de kaggle y ubicándolo en el directorio de instalación de su paquete kaggle. Sin embargo, si este método no funciona para usted, puede intentar cargar manualmente el archivo "co_properties.csv" descargado del enlace anterior, y adjunto al presente proyecto.

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
import sys
import kaggle
from zipfile import ZipFile
from kaggle.api.kaggle_api_extended import KaggleApi
import os
from IPython.display import display, HTML

def displaydf(df):
    display(HTML(df.to_html())) 

%matplotlib inline

destiny = os.path.join(os.path.expanduser('~'),'Documents\Properties_price\data')
if not os.path.exists(destiny):
    os.mkdir(destiny)
    
path_file = os.path.join(os.path.expanduser('~'), 'Documents\Properties_price\data\colombia-housing-properties-price.zip')
api = KaggleApi()
api.authenticate()
api.dataset_download_files('julianusugaortiz/colombia-housing-properties-price',path=destiny)

with ZipFile(path_file, 'r') as zipObj:
    zipObj.extractall(destiny)

if os.path.exists(path_file):
    os.remove(path_file)
else:
    print("The file zip does not exist")

path_file = os.path.join(os.path.expanduser('~'), 'Documents\Properties_price\data\co_properties.csv')
df = pd.read_csv(path_file, encoding='utf8')
print("Visualizamos el dataset para tener una idea general de la información que contiene:\n")
displaydf(df.iloc[0:20,])

Visualizamos el dataset para tener una idea general de la información que contiene:



,id,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,l4,l5,l6,rooms,bedrooms,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type
0,Z5GURF86+s3KVdbvKdx4dQ==,Propiedad,2020-04-07,2020-05-22,2020-04-07,6.287127,-75.336540,Colombia,Antioquia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.000000e+07,COP,NaN,Sevende Finca en Chaparrel de San Visent,sevende finca mas 9 lotes en san visente vereda chaparral a 2 kilometos de la8 parimentada lotes des de 90 millones en adelante con escrituras con sus es planadiones al gunos tienen agua propia ven iescoje eltullo,Otro,Venta
1,EbOqfrqoJKUuVFzkBymDgA==,Propiedad,2020-04-07,2020-05-15,2020-04-07,6.287127,-75.336540,Colombia,Antioquia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.500000e+08,COP,NaN,Sevende Finca en San Visente An Tioquia 14 etaresa,sevende finca en san visente aprosimada mente 14 etareas con casa lus agua de acueduto y aguas propias con carretera sepiden 450 millones,Otro,Venta
2,4et4/CQ6/jiiA31QcGbBSQ==,Propiedad,2020-04-07,2020-05-22,2020-04-07,NaN,NaN,Colombia,Antioquia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.600000e+09,COP,NaN,Venta de Lote Vereda Puente Pelaez El Retiro _ wasi1567707,"Lote de 145.336 metros, topografia quebrada, con nacimiento de agua, Lote ubicado en la Vereda Puente Pelaez a solo 15 minutos del parque principal de El Retiro. Precio $ 0 POSIBILIDAD DE VENTA DE MENOR AREA.",Otro,Venta
3,DnzyLOD2CU/exv0dQhVS/A==,Propiedad,2020-04-07,2020-07-02,2020-04-07,6.291447,-75.338812,Colombia,Antioquia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.500000e+07,COP,NaN,Lote/terreno de 7000 mts2 nacimiento de agua San Vicente Antioquia,"DESCRIPCION\n\nEspectacular oportunidad de negocio, para invertir o hacer una casa de recreo, fácil acceso para luz, agua y licencia de construccion.\n\nntra cualquier carro, a solo 10 minutos de San Vicente Antioquia y una hora y 10 minutos de Medellín, se pueden sacar dos banqueos, papeles al día, vista privilegiada al pueblo y 360%. \n\nEn este momento está en bosque y solo entra carro hasta donde empieza el lote, tiene un nacimiento de agua y pasa un arrollo por un lado, la zona es muy segura y de facil acceso.\n\nPara mas informacion 31161519y46 Alberto",Otro,Venta
4,Pg12IF9sRDSCcWZU6L2yig==,Propiedad,2020-04-07,2020-07-20,2020-04-07,3.457576,-76.558938,Colombia,Valle del Cauca,Cali,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.700000e+08,COP,NaN,"322.4 Venta de Lote en Aguacatal, Oeste de Cali","EXCELENTE OPORTUNIDAD PARA INVERSIÓN.\n\nVenta de Lote de 1200 m2 en El Aguacatal.\n\nCuenta con excelente ubicación, cerca del Colegio de la Presentación, excelente vista, zonas residenciales, paradas de transporte público, miradores y vías principales.",Otro,Venta
5,uh8DiLbc3HN7vTeT593MjQ==,Propiedad,2020-04-07,2020-07-20,2020-04-07,3.448069,-76.539430,Colombia,Valle del Cauca,Cali,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.700000e+08,COP,NaN,"1339.4 Venta de Lote Esquinero en San Antonio, Oeste de Cali","¡Excelente oportunidad de negocio!\n\nVenta de Lote de 240 mts2 en vehicular en San Antonio, Cali Valle del Cauca.\n\nSe encuentra central en la ciudad de alta valorización, sobre una avenida principal en un sector turístico cultural muy reconocido de la ciudad, cerca de iglesia y del parque San Antonio, hoteles, restaurantes, paradas de transporte, centros comerciales, entre otros.\n\nValor negociable.",Otro,Venta
6,vTQjNBLvIxPnkiUA20VS2A==,Propiedad,2020-04-07,2020-07-20,2020-04-07,NaN,NaN,Colombia,Antioquia,Bello,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.300000e+08,COP,NaN,Se vende lote,Se vende lote entre machado y Copacabana muy bn ubicado dentra carro y moto excelente vista agua propia para hacer lago tiene 3 metros de ancho por 30 de fondo precio negociable.,Otro,Venta
7,vNaJJcfDYZ32UD+a0EZpeg==,Propiedad,2020-04-07,2020-07-02,2020-04-07,6.338954,-75.541284,Colombia,Antioquia,Bello,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.620000e+08,COP,NaN,Venta de apartamento en obra gris Niquia Bello,"Cerca a: supermercados cercanos co

In [15]:
print("El dataset contiene {} filas y {} columnas\n".format(df.shape[0],df.shape[1]))
print("Los nombres de columnas en el data set son:\n {}\n".format(list(df.columns)))
print("Este dataset contiene información para los siguientes tipos de propiedades:\n {}\n".format(df.property_type.unique()))
print("Esta es una descripción general de las variables numéricas contenidas en el dataset:\n")
df.describe()

El dataset contiene 1000000 filas y 25 columnas

Los nombres de columnas en el data set son:
 ['id', 'ad_type', 'start_date', 'end_date', 'created_on', 'lat', 'lon', 'l1', 'l2', 'l3', 'l4', 'l5', 'l6', 'rooms', 'bedrooms', 'bathrooms', 'surface_total', 'surface_covered', 'price', 'currency', 'price_period', 'title', 'description', 'property_type', 'operation_type']

Este dataset contiene información para los siguientes tipos de propiedades:
 ['Otro' 'Apartamento' 'Lote' 'Oficina' 'Local comercial' 'Casa' 'Finca'
 'Depósito' 'Parqueadero' 'PH']

Esta es una descripción general de las variables numéricas contenidas en el dataset:



,lat,lon,rooms,bedrooms,bathrooms,surface_total,surface_covered,price
count,838824.000000,838824.000000,189968.000000,273655.000000,827253.000000,152765.000000,1.591200e+05,9.948770e+05
mean,5.967509,-74.948342,3.121094,2.687698,2.477456,996.887932,2.488659e+03,4.509809e+08
std,2.264294,1.076494,1.703937,14.270783,1.444242,6895.087600,3.618313e+05,2.123343e+09
min,-4.205429,-81.730319,1.000000,0.000000,1.000000,-36.000000,1.000000e+00,0.000000e+00
25%,4.662609,-75.598128,2.000000,2.000000,2.000000,66.000000,6.500000e+01,2.000000e+06
50%,5.059179,-75.442675,3.000000,3.000000,2.000000,100.000000,1.000000e+02,1.650000e+08
75%,6.339000,-74.066938,3.000000,3.000000,3.000000,210.000000,1.830000e+02,4.200000e+08
max,13.354000,-67.482570,40.000000,6820.000000,20.000000,200000.000000,1.323000e+08,8.500000e+11


# METODOLOGÍA PARA ABORDAR EL PROBLEMA

Para garantizar un trabajo efectivo sobre estos datos, usaremos la metodología CRIPS-DM quien nos guiará en la respuesta
cuestiones de interés sobre los datos que estamos estudiando. La metodología consta de los siguientes pasos:

1. Entendimiento del negocio
2. Entendimiento de los datos
3. Preparación de datos
4. Modelado de datos
5. Evaluación de resultados
6. Despliegue

La etapa de despliegue usualmente involucra la disponibilidad de la solución analítica en un ambiente de producción, como apoyo a la toma de decisiones, esto está fuera del alcance del presente cuaderno, pero intentaremos abordar sistemáticamente las fases hasta donde nos sea posible.

## 1. Entendimiento del negocio

¿Cuál es la cuestión principal que debemos resolver frente a los datos de propiedades inmobiliarias en colombia?. Esto naturalmente no tiene una única respuesta, sin embargo, para lograr un entendimiento de la situación podrían plantearse las siguientes cuestiones:

- **Cuestion 1.** ¿Presenta el precio de las propiedades inmobiliarias diferencias significativas entre diferentes regiones del país?
- **Cuestion 2.** ¿Hay una forma natural de agrupar propiedades según sus características?
- **Cuestion 2.** ¿Cuáles variables de las que tenemos disponibles nos permiten predecir de mejor manera el precio de una propiedad y qué tanta precisión se logra?

## 2. Entendimiento de los datos

Planteadas las cuestiones de interés vamos a dar una mirada más cercana a nuestros datos para ir respondiendo cada pregunta. En primer lugar revisaremos las variables que nos informan sobre la región donde se encuentran las propiedades.

In [ ]:

df.describe()

In [ ]:
# The above are variables that python is treating as numeric variables, and therefore, we 
# could send them into our linear model blindly to predict the response
# Let's take a quick look at our data first

df.hist();

In [ ]:
sns.heatmap(df.corr(), annot=True, fmt=".2f");

In [ ]:
# Here we can see that none of our variables appear to greatly correlated with salary
# and we can see that if someone was given an expected salary question, they either
# never answered the salary question or they were not given the salary question


# We an still go ahead and make predictions using these variables as a reminder of the 
# scikit learn way of fitting models.  The process is similar to quickly fit models of 
# all types - usually a four step process of - instantiate, fit, predict, score
# In most cases, we also will want to split data into training and test data to assure 
# we are not building models that overfit the data and do not extend well to new situations.

X = df[['CareerSatisfaction', 'HoursPerWeek', 'JobSatisfaction', 'StackOverflowSatisfaction']]
y = df['Salary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state=42)

lm_model = LinearRegression(normalize=True) # Here you could set any hyperparameters of your model
lm_model.fit(X_train, y_train) # If this model was to predict for new individuals, we probably would want
               # worry about train/test splits and cross-validation, but for now I am most 
               # interested in finding a model that just fits all of the data well

In [ ]:
### Notice the above breaks because of the NaN values, so we either need to fill or remove them
# Or we could write a conditional model that fits differently 
# depending on the values that are missing - we can see the nans based on the describe above
df.shape


#________ Video 1 through here on introduction to the data - could do a bit more EDA ________#

In [ ]:
### The easiest way to move onto a conclusion in a first pass is probably just with dropping

num_vars = df[['Salary', 'CareerSatisfaction', 'HoursPerWeek', 'JobSatisfaction', 'StackOverflowSatisfaction']]
df_dropna = num_vars.dropna(axis=0)

X = df_dropna[['CareerSatisfaction', 'HoursPerWeek', 'JobSatisfaction', 'StackOverflowSatisfaction']]
y = df_dropna['Salary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state=42)

lm_model = LinearRegression(normalize=True) # Here you could set any hyperparameters of your model
lm_model.fit(X_train, y_train) # If this model was to predict for new individuals, we probably would want
               # worry about train/test splits and cross-validation, but for now I am most 
               # interested in finding a model that just fits all of the data well

        
y_test_preds = lm_model.predict(X_test) #We can then use our fitted model to predict the salary for each
                                        #indvidual in our test set, and see how well these predictions
                                        #match the truth.

print(r2_score(y_test, y_test_preds)) #In this case we are predicting a continuous, numeric response.  Therefore, common
print(mean_squared_error(y_test, y_test_preds)) #metrics to assess fit include Rsquared and MSE.

In [ ]:
# Whoop - we built a model that predicts... but we are missing by ALOT!
# We can get a quick glimpse of how bad our predictions are...
# This suggests that 3% of the variability in salaries can be explained by these variables...
df_dropna.shape # But it also reduced our dataset to only 5338 rows 
                # ~20% of the original dataset size

# Recorded from here up


# Screencasts Remaining:
1. Imputation - first results
2. Categorical Variables - improved results, but what is happening?
3. Combat Overfitting - one method

In [ ]:
preds_vs_act = pd.DataFrame(np.hstack([y_test.values.reshape(y_test.size,1), y_test_preds.reshape(y_test.size,1)]))
preds_vs_act.columns = ['actual', 'preds']
preds_vs_act['diff'] = preds_vs_act['actual'] - preds_vs_act['preds']
preds_vs_act.head()

In [ ]:
### We can plot how far our predictions are from the actual values compaired to the
### predicted values - you can see that it isn't uncommon for us to miss salaries by
### 150000 and the overpredictions tend to be much worse than the underpredictions
### THere also appears to be a trend where our differences decrease as the predicted
### values increase on the test data.

plt.plot(preds_vs_act['preds'], preds_vs_act['diff'], 'bo');
plt.xlabel('predicted');
plt.ylabel('difference');

In [ ]:
#______Video 2 ____Our First Modeling Attempt (Mark all the bad things)________#



### There are tons of downfalls already - our predictions are pretty poor, we have predictions
### for only 20% of the total values that actually hold salaries, and we are only using 
### quantitative variables to predict.

### Given how bad the predictions are, we might not hurt anything by just filling the missing 
### values to make more predictions.

#Here we fill on the column means
df_fillna = num_vars.apply(lambda x: x.fillna(x.mean()),axis=0)

X = df_fillna[['CareerSatisfaction', 'HoursPerWeek', 'JobSatisfaction', 'StackOverflowSatisfaction']]
y = df_fillna['Salary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state=42)

lm_model = LinearRegression(normalize=True) # Here you could set any hyperparameters of your model
lm_model.fit(X_train, y_train) # If this model was to predict for new individuals, we probably would want
               # worry about train/test splits and cross-validation, but for now I am most 
               # interested in finding a model that just fits all of the data well

        
y_test_preds = lm_model.predict(X_test) #We can then use our fitted model to predict the salary for each
                                        #indvidual in our test set, and see how well these predictions
                                        #match the truth.

print(r2_score(y_test, y_test_preds)) #In this case we are predicting a continuous, numeric response.  Therefore, common


In [ ]:
X.shape

In [ ]:
### Now we can predict on everything, but our predictions are even worse!

preds_vs_act = pd.DataFrame(np.hstack([y_test.values.reshape(y_test.size,1), y_test_preds.reshape(y_test.size,1)]))
preds_vs_act.columns = ['actual', 'preds']
preds_vs_act['diff'] = preds_vs_act['actual'] - preds_vs_act['preds']
preds_vs_act.head()

In [ ]:
plt.plot(preds_vs_act['preds'], preds_vs_act['diff'], 'bo');
plt.xlabel('predicted');
plt.ylabel('difference');

In [ ]:
plt.plot(preds_vs_act['preds'], preds_vs_act['actual'], 'bo');
plt.xlabel('predicted');
plt.ylabel('actual'); #This looks less compelling that we are predicting well...
# I also think I found the mean amount...which aren't real 'actual' salaries

In [ ]:
### Some strange line here - probably because we filled in our average for everything
### Which was actually data leakage.  We shouldn't have done this at all. We would likely
### Have to use the mean of the old data to fill in the missing of the future data...

### But this does depend a bit - if on future homes, you will have the x-variables before
### having to predict, this really isn't data leakage, as you would have the abiltiy to update
### the inputed means with each new individual in your dataset.

### Really the values that have the mean value for the salary should be dropped - because
### those are not true salaries.

df_fillna = df_fillna.drop(df_fillna[df_fillna['Salary'] == np.mean(df['Salary'])].index)
df_fillna.shape # that's better. we only have this many non-null salaries in our original dataset


In [ ]:
#Below you can fit a new model with the missing salaries removed

In [ ]:
X = df_fillna[['CareerSatisfaction', 'HoursPerWeek', 'JobSatisfaction', 'StackOverflowSatisfaction']]
y = df_fillna['Salary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state=42)

lm_model = LinearRegression(normalize=True) # Here you could set any hyperparameters of your model
lm_model.fit(X_train, y_train) # If this model was to predict for new individuals, we probably would want
               # worry about train/test splits and cross-validation, but for now I am most 
               # interested in finding a model that just fits all of the data well

        
y_test_preds = lm_model.predict(X_test) #We can then use our fitted model to predict the salary for each
                                        #indvidual in our test set, and see how well these predictions
                                        #match the truth.

print(r2_score(y_test, y_test_preds)) #In this case we are predicting a continuous, numeric response.  Therefore, common
print(mean_squared_error(y_test, y_test_preds)) #metrics to assess fit include Rsquared and MSE.  


In [ ]:
##### Stop Video 2



### Now we can predict on everything, but our predictions are even worse!

preds_vs_act = pd.DataFrame(np.hstack([y_test.values.reshape(y_test.size,1), y_test_preds.reshape(y_test.size,1)]))
preds_vs_act.columns = ['actual', 'preds']
preds_vs_act['diff'] = preds_vs_act['actual'] - preds_vs_act['preds']
preds_vs_act.shape

In [ ]:
plt.plot(preds_vs_act['preds'], preds_vs_act['diff'], 'bo');
plt.xlabel('predicted');
plt.ylabel('difference');

In [ ]:
### When we see fan like shapes in the residual plots like this - it often suggests
### we might make better predictions on the log of the response

plt.plot(preds_vs_act['preds'], preds_vs_act['actual'], 'bo');
plt.xlabel('predicted');
plt.ylabel('actual'); #there appears to be a slight positive trend like we would want to see

In [ ]:
#______Video 3 Fill in Missing values with the mean - why this is bad_______#


### Let's see how we might be able to use categorical variables in our models.
### Though you might try to do something smart to reduce the feature space of your
### x-matrix (like find curved relationships that exist in salary comparing across categories).
### It is probably easier to just blindly encode all of the categorical variables as dummy
### variables in our models.

cat_vars_int = df.select_dtypes(include=['object']).copy().columns
# http://pbpython.com/categorical-encoding.html

len(cat_vars_int)

In [ ]:
### Now that we have a list of all the dummy variables we might be interested in... 
### Let's dummy code them, so that we can use them in our machine learning models
### you can do this with pandas (get dummies) or with sklearn (one hot encoding)
### Feel free to use whatever you are comfortable with

In [ ]:
for var in  cat_vars_int:
    # for each cat add dummy var, drop original column
    df = pd.concat([df.drop(var, axis=1), pd.get_dummies(df[var], prefix=var, prefix_sep='_', drop_first=True)], axis=1)

df.describe()

In [ ]:


### Because we have more rows than number of variables, it is actually possible
### for us to build a model that uses all of the columns to predict the response...
### Whether this is actually a good idea or not is up for debate - let's maybe
### choose some variables that seem like they might be related to salary and go from there.

### You can also see that the nulls are still dropped after dummy encoding, which means
### we will again need to figure out what to do with rows where those values are null.
### It might be okay to just use the mode of the dataset to fill in those values - though
### in reality, a lack of answer is maybe an indication that your answer is different 
### from the group and therefore, you didn't want to answer the question.

### We know there are 12891 non-NaN salaries to predict based on the previous model - so we
### want to make sure we can predict all of these salaries with our new model as well, but now
### unlike the 5 columns we had to choose from before we have more than 40,000 to choose from.
### This could be a great place for some PCA or PLS, but I would like to try and keep 
### the interpretability of the features as much as possible... so I am just going to
### use the original features. 

### We could try even adding interactions or other combinations of these features, but again
### this would make our features less interpretable. So you have to weigh the pros and cons
### of adding these features.

In [ ]:
df_result = pd.concat([df, df_fillna], axis=1, join='inner')
df_result.shape

In [ ]:
df_result['Salary'].head()['Salary']

In [ ]:
df_result = df_result.iloc[:,~df_result.columns.duplicated()]

In [ ]:
df_result.shape

In [ ]:
### Now we have no duplicated columns, we can focus on which of our new columns (and the 
### previously used columns) we would like to use to try and predict the response.  We might
### just go based on intuition, or we could try to find the variables that are most correlated
### Don't get too high of hopes - having a quant variable correlated with a 1-0 variable
### is not really what correlation coefficients are designed to detect.  They are meant
### to find linear relationships between quant variables. Though correlations are not built for
### finding these relations - they can still give a sense of which variables are best related


### Actually if you try to build the correlation matrix... it might run for a long time, and
### not be very legible anyway... Let's just fit some stuff that seems interesting 
### and intuitive.

In [ ]:
### Given how many columns we have to use - let's just drop all of the columns that have any
### missing values

df_result = df_result.dropna(axis=1, how='any')

In [ ]:
df_result.shape # which is only 6, sooo that kind of sucks at narrowing down this mess...

In [ ]:
y = df_result['Salary']
X = df_result.drop(['Respondent', 'Salary'], axis=1)

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state=42)

#lm_model = LinearRegression(normalize=True) # Here you could set any hyperparameters of your model
#lm_model.fit(X_train, y_train) # If this model was to predict for new individuals, we probably would want
               # worry about train/test splits and cross-validation, but for now I am most 
               # interested in finding a model that just fits all of the data well

        
#y_test_preds = lm_model.predict(X_test) #We can then use our fitted model to predict the salary for each
                                        #indvidual in our test set, and see how well these predictions
                                        #match the truth.

#print(r2_score(y_test, y_test_preds)) #In this case we are predicting a continuous, numeric response.  Therefore, common
#print(mean_squared_error(y_test, y_test_preds)) #metrics to assess fit include Rsquared and MSE.  
## Filling in the missing values does appear to have helped based on a preliminary check


In [ ]:
#print(r2_score(y_train, lm_model.predict(X_train)))
#print(mean_squared_error(y_train, lm_model.predict(X_train))) # What does this mean?

To combat the overfitting we have a number of options, but one way that would also reduce our run time would be to remove columns from our dataframe.  You will notice that sklearn does not provide pvals back for our coefficients, but it performs ridge regression by default.  So, therefore, we can consider that columns that have larger coefficients are also more useful for predicting our response variable.  How large is large enough to consider keeping? Well, that is a great question, and I also don't have a great answer...  We can try some stuff and see what works. 

Then we can also run cross-validation and aggregate our results to combat the overfitting we saw earlier using this reduced X matrix.

In [ ]:
# You could deal with these rare events in different ways - you could consider them as great predictors
# I am going to remove them - as I feel like they are likely not that indicative of other individuals
# I want to find overriding truths about the individuals who receive particular salaries.
# So, let's only consider columns where there are more than 1000 of the level of interest in the column.

reduce_X = X.iloc[:, np.where((X.sum() > 10) == True)[0]]
reduce_X.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(reduce_X, y, test_size = .30, random_state=42)

lm_model = LinearRegression(normalize=True) # Here you could set any hyperparameters of your model
lm_model.fit(X_train, y_train) # If this model was to predict for new individuals, we probably would want
               # worry about train/test splits and cross-validation, but for now I am most 
               # interested in finding a model that just fits all of the data well

        
y_test_preds = lm_model.predict(X_test) #We can then use our fitted model to predict the salary for each
                                        #indvidual in our test set, and see how well these predictions
                                        #match the truth.

print(r2_score(y_test, y_test_preds)) #In this case we are predicting a continuous, numeric response.  Therefore, common
print(mean_squared_error(y_test, y_test_preds)) #metrics to assess fit include Rsquared and MSE.  
## Filling in the missing values does appear to have helped based on a preliminary check

In [ ]:
print(r2_score(y_train, lm_model.predict(X_train))) #In this case we are predicting a continuous, numeric response.  Therefore, common
print(mean_squared_error(y_train, lm_model.predict(X_train))) #metrics to assess fit include Rsquared and MSE.  
## Filling in the missing values does appear to have helped based on a preliminary check

In [ ]:
### Let's see what be the best number of features to use based on the test set performance
def find_optimal_lm_mod(X, y, cutoffs, test_size = .30, random_state=42, plot=True):
    '''
    INPUT
    X - pandas dataframe, X matrix
    y - pandas dataframe, response variable
    cutoffs - list of ints, cutoff for number of non-zero values in dummy categorical vars
    test_size - float between 0 and 1, default 0.3, determines the proportion of data as test data
    random_state - int, default 42, controls random state for train_test_split
    plot - boolean, default 0.3, True to plot result
    
    OUTPUT
    r2_scores_test - list of floats of r2 scores on the test data
    r2_scores_train - list of floats of r2 scores on the train data
    lm_model - model object from sklearn
    X_train, X_test, y_train, y_test - output from sklearn train test split used for optimal model
    '''
    r2_scores_test, r2_scores_train, num_feats, results = [], [], [], dict()
    for cutoff in cutoffs:
        
        #reduce X matrix
        reduce_X = X.iloc[:, np.where((X.sum() > cutoff) == True)[0]]
        num_feats.append(reduce_X.shape[1])

        #split the data into train and test
        X_train, X_test, y_train, y_test = train_test_split(reduce_X, y, test_size = test_size, random_state=random_state)

        #fit the model and obtain pred response
        lm_model = LinearRegression(normalize=True) 
        lm_model.fit(X_train, y_train)
        y_test_preds = lm_model.predict(X_test)
        y_train_preds = lm_model.predict(X_train)
        
        #append the r2 value from the test set
        r2_scores_test.append(r2_score(y_test, y_test_preds))
        r2_scores_train.append(r2_score(y_train, y_train_preds))
        results[str(cutoff)] = r2_score(y_test, y_test_preds)
    
    if plot:
        plt.plot(num_feats, r2_scores_test, label="Test", alpha=.5)
        plt.plot(num_feats, r2_scores_train, label="Train", alpha=.5)
        plt.xlabel('Number of Features')
        plt.ylabel('Rsquared')
        plt.title('Rsquared by Number of Features')
        plt.legend(loc=1)
        plt.show()
        
    best_cutoff = max(results, key=results.get)
    
    #reduce X matrix
    reduce_X = X.iloc[:, np.where((X.sum() > int(best_cutoff)) == True)[0]]
    num_feats.append(reduce_X.shape[1])

    #split the data into train and test
    X_train, X_test, y_train, y_test = train_test_split(reduce_X, y, test_size = test_size, random_state=random_state)

    #fit the model
    lm_model = LinearRegression(normalize=True) 
    lm_model.fit(X_train, y_train)
        
    return r2_scores_test, r2_scores_train, lm_model, X_train, X_test, y_train, y_test

In [ ]:
cutoffs = [5000, 3500, 2500, 1000, 100, 50, 30, 20, 10, 5]
r2_scores_test, r2_scores_train, lm_model, X_train, X_test, y_train, y_test = find_optimal_lm_mod(X, y, cutoffs)

In [ ]:
#______Video 4 Creating Dummy Variables & Other Alternatives for Categorical Variables____#




### Now that we have the best model in terms of the r2 on the test data, we can use this model to see which features
### appear to be most important, and what impact they have on salary.

X_train.shape # we have 1081 features in the optimal model - let's look at some of them


In [ ]:
y_test_preds = lm_model.predict(X_test)

preds_vs_act = pd.DataFrame(np.hstack([y_test.values.reshape(y_test.size,1), y_test_preds.reshape(y_test.size,1)]))
preds_vs_act.columns = ['actual', 'preds']
preds_vs_act['diff'] = preds_vs_act['actual'] - preds_vs_act['preds']

plt.plot(preds_vs_act['preds'], preds_vs_act['diff'], 'bo');
plt.xlabel('predicted');
plt.ylabel('difference');

In [ ]:
plt.plot(preds_vs_act['preds'], preds_vs_act['actual'], 'bo');
plt.xlabel('predicted');
plt.ylabel('actual'); #there appears to be a slight positive trend like we would want to see

In [ ]:
coefs_df = pd.DataFrame()

coefs_df['est_int'] = X_train.columns
coefs_df['coefs'] = lm_model.coef_
coefs_df['abs_coefs'] = np.abs(lm_model.coef_)

coefs_df.sort_values('abs_coefs', ascending=False).head(20)

In [ ]:
lm_model.intercept_

In [ ]:
X_train.shape, sum(X_train['Professional_Professional developer'])


#_____Video 7 Interpretting the results_____#

In [ ]:
#____Video 8 - Ensemble Models______#

### One of the best out of the box methods for supervised machine learning
### is known as the RandomForest - let's see if we can use this model to outperform
### The linear model from earlier.

from sklearn.ensemble import RandomForestRegressor

### Let's see what be the best number of features to use based on the test set performance
def find_optimal_rf_mod(X, y, cutoffs, test_size = .30, random_state=42, plot=True):
    '''
    INPUT
    X - pandas dataframe, X matrix
    y - pandas dataframe, response variable
    cutoffs - list of ints, cutoff for number of non-zero values in dummy categorical vars
    test_size - float between 0 and 1, default 0.3, determines the proportion of data as test data
    random_state - int, default 42, controls random state for train_test_split
    plot - boolean, default 0.3, True to plot result
    kwargs - include the arguments you want to pass to the rf model
    
    OUTPUT
    r2_scores_test - list of floats of r2 scores on the test data
    r2_scores_train - list of floats of r2 scores on the train data
    rf_model - model object from sklearn
    X_train, X_test, y_train, y_test - output from sklearn train test split used for optimal model
    '''
    r2_scores_test, r2_scores_train, num_feats, results = [], [], [], dict()
    for cutoff in cutoffs:
        
        #reduce X matrix
        reduce_X = X.iloc[:, np.where((X.sum() > cutoff) == True)[0]]
        num_feats.append(reduce_X.shape[1])

        #split the data into train and test
        X_train, X_test, y_train, y_test = train_test_split(reduce_X, y, test_size = test_size, random_state=random_state)

        #fit the model and obtain pred response

        rf_model = RandomForestRegressor()  #no normalizing here, but could tune other hyperparameters
        rf_model.fit(X_train, y_train)
        y_test_preds = rf_model.predict(X_test)
        y_train_preds = rf_model.predict(X_train)
        
        #append the r2 value from the test set
        r2_scores_test.append(r2_score(y_test, y_test_preds))
        r2_scores_train.append(r2_score(y_train, y_train_preds))
        results[str(cutoff)] = r2_score(y_test, y_test_preds)
    
    if plot:
        plt.plot(num_feats, r2_scores_test, label="Test", alpha=.5)
        plt.plot(num_feats, r2_scores_train, label="Train", alpha=.5)
        plt.xlabel('Number of Features')
        plt.ylabel('Rsquared')
        plt.title('Rsquared by Number of Features')
        plt.legend(loc=1)
        plt.show()
        
    best_cutoff = max(results, key=results.get)
    
    #reduce X matrix
    reduce_X = X.iloc[:, np.where((X.sum() > int(best_cutoff)) == True)[0]]
    num_feats.append(reduce_X.shape[1])

    #split the data into train and test
    X_train, X_test, y_train, y_test = train_test_split(reduce_X, y, test_size = test_size, random_state=random_state)

    #fit the model
    rf_model = RandomForestRegressor() 
    rf_model.fit(X_train, y_train)
        
    return r2_scores_test, r2_scores_train, rf_model, X_train, X_test, y_train, y_test

In [ ]:
cutoffs = [5000, 3500, 2500, 1000, 100, 50, 30, 20, 10, 5]
r2_test, r2_train, rf_model, X_train, X_test, y_train, y_test = find_optimal_rf_mod(X, y, cutoffs)

In [ ]:
y_test_preds = rf_model.predict(X_test)

preds_vs_act = pd.DataFrame(np.hstack([y_test.values.reshape(y_test.size,1), y_test_preds.reshape(y_test.size,1)]))
preds_vs_act.columns = ['actual', 'preds']
preds_vs_act['diff'] = preds_vs_act['actual'] - preds_vs_act['preds']

plt.plot(preds_vs_act['preds'], preds_vs_act['diff'], 'bo');
plt.xlabel('predicted');
plt.ylabel('difference');

In [ ]:
#Looks like this overfits quite a bit... 

In [ ]:
from sklearn.model_selection import GridSearchCV

### Let's see what be the best number of features to use based on the test set performance
def find_optimal_rf_mod(X, y, cutoffs, test_size = .30, random_state=42, plot=True, param_grid=None):
    '''
    INPUT
    X - pandas dataframe, X matrix
    y - pandas dataframe, response variable
    cutoffs - list of ints, cutoff for number of non-zero values in dummy categorical vars
    test_size - float between 0 and 1, default 0.3, determines the proportion of data as test data
    random_state - int, default 42, controls random state for train_test_split
    plot - boolean, default 0.3, True to plot result
    kwargs - include the arguments you want to pass to the rf model
    
    OUTPUT
    r2_scores_test - list of floats of r2 scores on the test data
    r2_scores_train - list of floats of r2 scores on the train data
    rf_model - model object from sklearn
    X_train, X_test, y_train, y_test - output from sklearn train test split used for optimal model
    '''

    r2_scores_test, r2_scores_train, num_feats, results = [], [], [], dict()
    for cutoff in cutoffs:

        #reduce X matrix
        reduce_X = X.iloc[:, np.where((X.sum() > cutoff) == True)[0]]
        num_feats.append(reduce_X.shape[1])

        #split the data into train and test
        X_train, X_test, y_train, y_test = train_test_split(reduce_X, y, test_size = test_size, random_state=random_state)

        #fit the model and obtain pred response
        if param_grid==None:
            rf_model = RandomForestRegressor()  #no normalizing here, but could tune other hyperparameters

        else:
            rf_inst = RandomForestRegressor(n_jobs=-1, verbose=1)
            rf_model = GridSearchCV(rf_inst, param_grid, n_jobs=-1) 
            
        rf_model.fit(X_train, y_train)
        y_test_preds = rf_model.predict(X_test)
        y_train_preds = rf_model.predict(X_train)

        #append the r2 value from the test set
        r2_scores_test.append(r2_score(y_test, y_test_preds))
        r2_scores_train.append(r2_score(y_train, y_train_preds))
        results[str(cutoff)] = r2_score(y_test, y_test_preds)

    if plot:
        plt.plot(num_feats, r2_scores_test, label="Test", alpha=.5)
        plt.plot(num_feats, r2_scores_train, label="Train", alpha=.5)
        plt.xlabel('Number of Features')
        plt.ylabel('Rsquared')
        plt.title('Rsquared by Number of Features')
        plt.legend(loc=1)
        plt.show()
        
    best_cutoff = max(results, key=results.get)

    #reduce X matrix
    reduce_X = X.iloc[:, np.where((X.sum() > int(best_cutoff)) == True)[0]]
    num_feats.append(reduce_X.shape[1])

    #split the data into train and test
    X_train, X_test, y_train, y_test = train_test_split(reduce_X, y, test_size = test_size, random_state=random_state)

    #fit the model
    if param_grid==None:
        rf_model = RandomForestRegressor()  #no normalizing here, but could tune other hyperparameters

    else:
        rf_inst = RandomForestRegressor(n_jobs=-1, verbose=1)
        rf_model = GridSearchCV(rf_inst, param_grid, n_jobs=-1) 
    rf_model.fit(X_train, y_train)
     
    return r2_scores_test, r2_scores_train, rf_model, X_train, X_test, y_train, y_test

In [ ]:
cutoffs = [5000, 3500, 2500, 1000, 100, 50, 30, 20, 10, 5]
params = {'n_estimators': [10, 100, 1000], 'max_depth': [1, 5, 10, 100]}
r2_test, r2_train, rf_model, X_train, X_test, y_train, y_test = find_optimal_rf_mod(X, y, cutoffs, param_grid=params)